In [125]:
import uproot
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import math
from scipy.fft import fft, fftfreq
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import os
from plotly.subplots import make_subplots
from plotly import tools
import plotly.offline as pyo
import sys
from tqdm import tqdm

In [126]:
#Opens log files that contain interaction information
#with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_nu.log') as file:
#    lines_nu = [line_nu.rstrip() for line_nu in file]
with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_nu_all_planes.log') as file:
    lines_nu = [line_nu.rstrip() for line_nu in file]
#with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_data.log') as file:
#    lines_data = [line_data.rstrip() for line_data in file]
with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_data_allplanes.log') as file:
    lines_data = [line_data.rstrip() for line_data in file]
for i in range(10):
    print(lines_nu[i][0])

S
S
S
S
S
S
S
S
S
S


In [129]:
#Dictionaries to be filled in interaction information
#Signal is true interactions matched with reconstructed intercations
df_signal_nu = {'Type': [], 'Run': [], 'Evt': [], 'Subrun':[],'File':[],'Nu_id':[],'Image_id':[],'Interaction_id':[], 'Category':[],
     'Category_topology':[],'category_interaction_mode':[],'leading_electron_ke_true':[], 'leading_electron_ke_reco':[],
     'leading_proton_ke_true':[],'leading_proton_ke_reco':[],'leading_electron_pid_true':[], 'leading_electron_pid_reco':[],
     'leading_proton_pid_true':[],'leading_proton_pid_reco':[],'visible_energy_true':[],'visible_energy_reco':[],'leading_electron_pt_true':[],
     'leading_electron_pt_reco':[],'leading_proton_pt_true':[],'leading_proton_pt_reco':[],'electron_angle_polar_angle_true':[],
     'leading_electron_polar_angle_reco':[],'electron_azimuthal_angle_true':[],'electron_azimuthal_angle_reco':[],
     'opening_angle_true':[],'opening_angle_reco':[],'interaction_pt_true':[],'interaction_pt_reco':[],
     'phiT_true':[],'phiT_reco':[],'alphaT_true':[],'alphaT_reco':[],'electron_softmax':[],'proton_softmax':[],
     'all_1e1p_cut':[],'all_1eNp_cut':[],'all_1eX_cut':[],'End':[]}

#Selected is reco interactions matched with true interactions
df_selected_nu =  {'Type': [], 'Run': [], 'Evt': [], 'Subrun':[],'File':[],'Nu_id':[],'Image_id':[],'Interaction_id':[], 'Category':[],
     'Category_topology':[],'category_interaction_mode':[],'leading_electron_ke_true':[], 'leading_electron_ke_reco':[],
     'leading_proton_ke_true':[],'leading_proton_ke_reco':[],'leading_electron_pid_true':[], 'leading_electron_pid_reco':[],
     'leading_proton_pid_true':[],'leading_proton_pid_reco':[],'visible_energy_true':[],'visible_energy_reco':[],'leading_electron_pt_true':[],
     'leading_electron_pt_reco':[],'leading_proton_pt_true':[],'leading_proton_pt_reco':[],'electron_angle_polar_angle_true':[],
     'leading_electron_polar_angle_reco':[],'electron_azimuthal_angle_true':[],'electron_azimuthal_angle_reco':[],
     'opening_angle_true':[],'opening_angle_reco':[],'interaction_pt_true':[],'interaction_pt_reco':[],
     'phiT_true':[],'phiT_reco':[],'alphaT_true':[],'alphaT_reco':[],'electron_softmax':[],'proton_softmax':[],
     'all_1e1p_cut':[],'all_1eNp_cut':[],'all_1eX_cut':[],'End':[]}

list_df_MC = ['Type', 'Run', 'Evt', 'Subrun','File','Nu_id','Image_id','Interaction_id', 'Category',
     'Category_topology','category_interaction_mode','leading_electron_ke_true', 'leading_electron_ke_reco',
     'leading_proton_ke_true','leading_proton_ke_reco','leading_electron_pid_true', 'leading_electron_pid_reco',
     'leading_proton_pid_true','leading_proton_pid_reco','visible_energy_true','visible_energy_reco','leading_electron_pt_true',
     'leading_electron_pt_reco','leading_proton_pt_true','leading_proton_pt_reco','electron_angle_polar_angle_true',
     'leading_electron_polar_angle_reco','electron_azimuthal_angle_true','electron_azimuthal_angle_reco',
     'opening_angle_true','opening_angle_reco','interaction_pt_true','interaction_pt_reco',
     'phiT_true','phiT_reco','alphaT_true','alphaT_reco','electron_softmax','proton_softmax','all_1e1p_cut','all_1eNp_cut','all_1eX_cut',"End"]


#Fills the dictionaries with data            
for line_nu in lines_nu:
    line_nu = line_nu.split(",")
    #print(line_nu[0])
    if line_nu[0] == 'SIGNAL':
        for i in range(len(line_nu)):
            df_signal_nu[list_df_MC[i]].append(line_nu[i])
    elif line_nu[0] == 'SELECTED':
        for i in range(len(line_nu)):
            df_selected_nu[list_df_MC[i]].append(line_nu[i])
print(lines_nu[0])
            
df_data= {'Type': [], 'Run': [], 'Evt': [], 'Subrun':[],'Image_id':[],'Interaction_id':[], 'leading_electron_ke_reco':[],
     'leading_proton_ke_reco':[],'visible_energy_reco':[],
     'leading_electron_pt_reco':[],'leading_proton_pt_reco':[],
     'leading_electron_polar_angle_reco':[],'electron_azimuthal_angle_reco':[],
     'opening_angle_reco':[],'interaction_pt_reco':[],
     'phiT_reco':[],'alphaT_reco':[],'electron_softmax':[],'proton_softmax':[],'all_1e1p_cut':[],'all_1eNp_cut':[],'all_1eX_cut':[],'End':[]}
list_df = ['Type', 'Run', 'Evt', 'Subrun','Image_id','Interaction_id', 'leading_electron_ke_reco',
     'leading_proton_ke_reco','visible_energy_reco',
     'leading_electron_pt_reco','leading_proton_pt_reco',
     'leading_electron_polar_angle_reco','electron_azimuthal_angle_reco',
     'opening_angle_reco','interaction_pt_reco',
     'phiT_reco','alphaT_reco','electron_softmax','proton_softmax','all_1e1p_cut','all_1eNp_cut','all_1eX_cut',"End"]
print(lines_data[0])
for line_data in lines_data:
    line_data = line_data.split(",")
    if line_data[0] == 'DATA':
        for i in range(len(line_data)):
            df_data[list_df[i]].append(line_data[i])      

SIGNAL,1,15,0,detsim_2d_icarus_fitFR_refactored_detsim_stage0_stage1_13566914_0.root,0,1.000000,0.000000,4.000000,1.000000,1.000000,1181.655871,696.524619,1181.655871,60.144124,1,1,1,4,26.144212,757.179741,239.751541,93.225227,239.751541,133.409027,0.204223,0.175223,0.928905,0.719149,0.000000,0.236422,239.751549,224.771718,-nan,-nan,3.141593,2.987979,0.756625,0.838597,1,1,1,
DATA,9794,38259,1,1.000000,0.000000,192.803552,-1.000000,193.314551,148.233551,148.233551,1.617815,2.035943,0.000345,148.233554,-nan,3.141593,0.750986,0.000000,0,0,1,


In [130]:
def MC_selection(df_selected_nu,variable):
    signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other = [],[],[],[],[],[],[],[]
    total_events =0
    for index, category in enumerate(df_selected_nu['Category_topology']):
        
        if df_selected_nu['all_1e1p_cut'][index] == '1' or df_selected_nu['all_1eNp_cut'][index] == '1':
            
            total_events +=1
            if (category[0] == '0' or category[0] == '2'):
                #print(df_selected_nu['File'][index])
                signal.append(float(df_selected_nu[variable][index]))
            elif category[0] == '1':
                sel_nue_other.append(float(df_selected_nu[variable][index]))
            elif category[0] == '3':
                sel_nue_other.append(float(df_selected_nu[variable][index]))
            elif category[0] == '4':
                sel_nue_other.append(float(df_selected_nu[variable][index]))
            elif category[0] == '5':
                sel_NC.append(float(df_selected_nu[variable][index]))
            elif category[0] == '6':
                sel_cosmic.append(float(df_selected_nu[variable][index]))
            elif category[0] == '7':
                sel_numu.append(float(df_selected_nu[variable][index]))
            else:
                #print()
                sel_other.append(float(df_selected_nu[variable][index]))
    return signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other, total_events

In [131]:

def data_selection(df_data,variable):
    data_1eNp = []
    for index,i in enumerate(df_data['all_1e1p_cut']):
        if int(i[0]) == 1:
            #if int(i[0]) == 0:
            data_1eNp.append(float(df_data[variable][index]))
    for index,i in enumerate(df_data['all_1eNp_cut']):
        if int(i[0]) == 1:
            #if int(i[0]) == 0:
            data_1eNp.append(float(df_data[variable][index]))
    return data_1eNp

In [132]:
def hist_1d(metric, bounds,bins, xaxis_name, Plot_title, selection):
    fig = make_subplots(rows=1,cols=1,subplot_titles = (f"<span style='font-size: 42px;'>{Plot_title}",))
    
    signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other,total_events = MC_selection(df_selected_nu,metric)
    
    print('Purity: ', len(signal)/total_events)
    print('Number of signal: ',len(signal),' Number of selected:', total_events)
    cosmic_size = len(sel_cosmic)
    numu_size = len(sel_numu)
    NC_size = len(sel_NC)
    signal_size = len(signal)
    nue_size = len(sel_nue_other)
    other_size = len(sel_other)
    
    
    fig.add_trace(go.Histogram(x=sel_cosmic,marker_color = 'yellow',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                               name = f'Cosmics ({cosmic_size}, {100*cosmic_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_numu,marker_color = 'blue',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>\u03BC</sub> ({numu_size}, {100*numu_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_NC,marker_color = 'black',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'NC ({NC_size}, {100*NC_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_other,marker_color = 'purple',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'Other ({other_size}, {100*other_size/total_events:.2f}%)'),row = 1, col =1)
    
    if selection != 'Nue':
        #nue_size =  len(df[metric][mask_other_nue][reco_signal][mask])
        fig.add_trace(go.Histogram(x=sel_nue_other,marker_color = 'orange',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                                  name = f'Other \u03BD<sub>e</sub> ({nue_size}, {100*nue_size/total_events:.2f}%)'),row = 1, col =1)
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'{selection} ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    else:
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>e</sub> ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    
    
    fig.update_xaxes(title_text = xaxis_name,row = 1, col = 1)
    fig.update_yaxes(title_text = "Entries",row = 1, col = 1,)
    fig.update_layout(font = dict(size=26),legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.43,
        bgcolor="LightSteelBlue",
            bordercolor="Black",
    ))
    fig.update_annotations(font_size=46)
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis = dict(range = bounds))
    fig.update_layout(height = 900, width = 900,showlegend = True)
    fig.show()


In [133]:
#hist_1d('reco_nu_energy',[0,5000],[0,5000,250],'Reco Neutrino Energy [MeV]','Selected 1e1p Events','1e1p')
hist_1d('visible_energy_reco',[0,6000],[0,6000,250],'Reco Neutrino Energy [MeV]','Selected 1eNp Events','1eNp')
#hist_1d('reco_nu_energy',[0,5000],[0,5000,100],'Reco Neutrino Energy [MeV]','Selected CC \u03BD<sub>e</sub> Inclusive Events','Nue')

Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


In [134]:
def get_stat_cov(bins):
    stat_cov = np.diag(bins)
    stat_err = np.sqrt(np.diagonal(stat_cov))
    return stat_cov, stat_err

In [135]:
def hist_1d_norm(metric, y_axis,bounds,bin_size, xaxis_name, Plot_title, selection):
    fig = make_subplots(rows=1,cols=1,subplot_titles = (Plot_title,))
 #signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other
    signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other,total_events = MC_selection(df_selected_nu,metric)
    data_1eNp = data_selection(df_data,metric)
    #print(data_1eNp)
    energy_limit = bounds[1]
    accuracy_sel,accuracy_1eNp,accuracy_nue=[],[],[]
    error_sel = {'high': [],'low': []} 
    error_1eNp={'high':[],'low':[]}
    error_nue={'high':[],'low':[]}
    counts=[]
    energy_bin = []
    print('Purity: ', len(signal)/total_events)
    print('Number of signal: ',len(signal),' Number of selected:', total_events)
    cosmic_size = len(sel_cosmic)
    numu_size = len(sel_numu)
    NC_size = len(sel_NC)
    signal_size = len(signal)
    nue_size = len(sel_nue_other)
    other_size = len(sel_other)
    data_sel = {'Data':[],'UnNormalized':[]}
    pur_sel = {'Cosmics':[],'\u03BD<sub>\u03BC</sub>':[],'NC':[],'Other \u03BD<sub>e</sub>':[],'Signal':[]}
    for b in range(int(energy_limit/bin_size)):
        data_sel['UnNormalized'].append(len([x for x in data_1eNp if x >=b*bin_size and x < (b+1)*bin_size]))
        #print(len([x for x in data_1eNp if x >=b*bin_size and x < (b+1)*bin_size])/len(data_1eNp))
        data_sel['Data'].append(len([x for x in data_1eNp if x >=b*bin_size and x < (b+1)*bin_size])/len(data_1eNp))
        pur_sel['Signal'].append(len([x for x in signal if x >=b*bin_size and x < (b+1)*bin_size])/total_events)
        pur_sel['Cosmics'].append(len([x for x in sel_cosmic if x >=b*bin_size and x < (b+1)*bin_size])/total_events)
        pur_sel['\u03BD<sub>\u03BC</sub>'].append(len([x for x in sel_numu if x >=b*bin_size and x < (b+1)*bin_size])/total_events)
        pur_sel['NC'].append(len([x for x in sel_NC if x >=b*bin_size and x < (b+1)*bin_size])/total_events)
        if selection != 'Nue':
            pur_sel['Other \u03BD<sub>e</sub>'].append(len([x for x in sel_1e if x >=b*bin_size and x < (b+1)*bin_size])/total_events)
            pur_sel['Other \u03BD<sub>e</sub>'][b]+=len([x for x in sel_1e1pi1p if x >=b*bin_size and x < (b+1)*bin_size])/total_events
            pur_sel['Other \u03BD<sub>e</sub>'][b]+=len([x for x in sel_nue_other if x >=b*bin_size and x < (b+1)*bin_size])/total_events

                
            
        energy_bin.append(((b+1)*bin_size)-(bin_size/2))
    bins_data = np.array(data_sel['UnNormalized'])
    stat_cov_data,stat_err_data = get_stat_cov(bins_data)
    stat_cov_data_frac = stat_cov_data / np.outer(bins_data, bins_data)
    scaledata = 1/np.sum(bins_data)
    bins_data_scaled = bins_data * scaledata
    stat_cov_data_scaled = np.outer(bins_data_scaled, bins_data_scaled) * stat_cov_data_frac
    stat_err_data_scaled = np.sqrt(np.diag(stat_cov_data_scaled))
    bins_data = bins_data_scaled
    stat_cov_data = stat_cov_data_scaled
    stat_err_data = stat_err_data_scaled
    #mask = (df['reco_electron_conversion_dist']<2) #& (df['reco_electron_energy'] > 80)
    #Create the masks for histograms
    fig.add_trace(go.Scatter(x=energy_bin, y=data_sel['Data'],marker_color = 'black',mode='markers',name = f'Data',error_x=dict(
            type='data',
            array=[bin_size/2]*len(energy_bin)),
            error_y=dict(
            type='data',
            visible=True,
            array=stat_err_data,)
            ),row = 1, col =1)
    color = ['yellow','blue','pink','orange','green','pink','cyan','lime','brown','gray','red']
    lists = [sel_cosmic,sel_numu,sel_NC,sel_nue_other,signal]
    for i,key in enumerate(pur_sel):
        fig.add_trace(go.Bar(x=energy_bin, y=pur_sel[key],width = [bin_size]*len(energy_bin),marker_color = color[i],name = f'{key}: {(len(lists[i])/total_events)*100:.2f}%'),row = 1, col =1)
    
    #fig.add_trace(go.Bar(x=energy_bin, y=eff_sel['signal'],width = [bin_size]*len(energy_bin),marker_color = 'green',name = 'Signal'),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['signal'],marker_color = 'green',name = 'Signal',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #if selection == '1e1p':
    #    fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['1eNp'],marker_color = 'black',name = '1eNp',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #if selection != "Nue Inclusive":
    #    fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['1e'],marker_color = 'yellow',name = '1e',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #    fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['1e1pi1p'],marker_color = 'blue',name = '1e1pi1p',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #    fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['nue'],marker_color = 'orange',name = 'nue',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['no_e'],marker_color = 'pink',name = 'no_e',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['e2g'],marker_color = 'cyan',name = 'e2g',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['Ne'],marker_color = 'lime',name = 'Ne',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['uncontained'],marker_color = 'brown',name = 'uncontained',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['flash'],marker_color = 'gray',name = 'flash',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    #fig.add_trace(go.Scatter(x=energy_bin, y=eff_sel['other'],marker_color = 'red',name = 'other',mode='markers',error_x=dict(type='data',array=[bin_size/2]*len(energy_bin))),row = 1, col =1)
    
    fig.update_layout(font = dict(size=26),legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.5,
        bgcolor="LightSteelBlue",
            bordercolor="Black",
    ))
    fig.update_xaxes(title_text = xaxis_name,row = 1, col = 1)
    fig.update_annotations(font_size=36)
    fig.update_yaxes(title_text = f"Entries Normalized ",row = 1, col = 1)
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis = dict(range = bounds))
    fig.update_layout(yaxis = dict(range = y_axis))
    fig.update_layout(height = 900, width = 900,showlegend = True)
    fig.show()


In [136]:
#hist_1d_norm('reco_nu_energy',[0,5000],250,'True Neutrino Inital Energy [MeV]','True 1e1p Events','1e1p')
hist_1d_norm('visible_energy_reco',[0,.32],[0,6000],250,'Reco Neutrino Inital Energy [MeV]','Selected 1eNP Events','1eNp')
hist_1d_norm('leading_electron_ke_reco',[0,.32],[0,6000],250,'Reco Electron Energy [MeV]','Selected 1eNP Events','1eNp')
hist_1d_norm('electron_softmax',[0,.4],[0.5,1],.025,'Electron Softmax Score','Selected 1eNP Events','1eNp')
hist_1d_norm('leading_proton_ke_reco',[0,.32],[0,800],50,'Reco Proton Energy [MeV]','Selected 1eNP Events','1eNp')
hist_1d_norm('proton_softmax',[0,.4],[0.4,1],.03,'Proton Softmax Score','Selected 1eNP Events','1eNp')

#hist_1d_norm('reco_nu_energy',[0,5000],250,'True Neutrino Inital Energy [MeV]','True Nue Inclusive Events','Nue Inclusive')

Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


/var/folders/s1/zjpdvkm93hz1tmp35vw_5k880000gn/T/ipykernel_10536/2426128653.py:42: RuntimeWarning:

invalid value encountered in divide



Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


In [137]:
hist_1d_norm('opening_angle_reco',[0,.25],[0,3.2],.2,'Opening angle','Selected 1eNP Events','1eNp')
hist_1d_norm('electron_azimuthal_angle_reco',[0,.4],[0,3.2],.2,'Electron Azimuthal Angle','Selected 1eNP Events','1eNp')
hist_1d_norm('leading_electron_polar_angle_reco',[0,.4],[0,3.2],.2,'Electron Polar Angle','Selected 1eNP Events','1eNp')

Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


In [138]:
hist_1d_norm('leading_electron_pt_reco',[0,.35],[0,2500],150,'Electron Tranverse Momentum','Selected 1eNP Events','1eNp')
hist_1d_norm('leading_proton_pt_reco',[0,.25],[0,1200],60,'Proton Tranverse Momentum','Selected 1eNP Events','1eNp')
hist_1d_norm('phiT_reco',[0,.25],[0,100],.1,'PhiT','Selected 1eNP Events','1eNp')
hist_1d_norm('alphaT_reco',[0,.25],[0,3.2],.2,'AlphaT','Selected 1eNP Events','1eNp')
#hist_1d_norm('leading_electron_polar_angle_reco',[0,.4],[0,3.2],.2,'Electron Polar Angle','True 1eNP Events','1eNp')
#hist_1d_norm('leading_electron_polar_angle_reco',[0,.4],[0,3.2],.2,'Electron Polar Angle','True 1eNP Events','1eNp')

Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


/var/folders/s1/zjpdvkm93hz1tmp35vw_5k880000gn/T/ipykernel_10536/2426128653.py:42: RuntimeWarning:

invalid value encountered in divide



Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609


/var/folders/s1/zjpdvkm93hz1tmp35vw_5k880000gn/T/ipykernel_10536/2426128653.py:43: RuntimeWarning:

divide by zero encountered in scalar divide

/var/folders/s1/zjpdvkm93hz1tmp35vw_5k880000gn/T/ipykernel_10536/2426128653.py:44: RuntimeWarning:

invalid value encountered in multiply



Purity:  0.7625854568054692
Number of signal:  1227  Number of selected: 1609
